<a href="https://colab.research.google.com/github/Hamze-alseed/Portfolio/blob/main/FaceID_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q insightface onnxruntime-gpu opencv-python scikit-learn


Setup

In [3]:
import cv2
import numpy as np
from insightface.app import FaceAnalysis
from sklearn.metrics.pairwise import cosine_similarity
from google.colab.patches import cv2_imshow
from IPython.display import display, Javascript
from google.colab.output import eval_js
import base64


Photo capture tool

In [4]:
def take_photo():
    js = Javascript('''
        async function takePhoto() {
          const div = document.createElement('div');
          const video = document.createElement('video');
          const stream = await navigator.mediaDevices.getUserMedia({video: true});
          document.body.appendChild(div);
          div.appendChild(video);
          video.srcObject = stream;
          await video.play();

          await new Promise((resolve) => setTimeout(resolve, 2000));

          const canvas = document.createElement('canvas');
          canvas.width = video.videoWidth;
          canvas.height = video.videoHeight;
          canvas.getContext('2d').drawImage(video, 0, 0);
          stream.getTracks()[0].stop();
          div.remove();

          return canvas.toDataURL('image/jpeg');
        }
        takePhoto();
    ''')
    display(js)
    data = eval_js("takePhoto()")
    binary = base64.b64decode(data.split(',')[1])
    img = cv2.imdecode(np.frombuffer(binary, np.uint8), cv2.IMREAD_COLOR)
    return img


Uploading insightface

In [ ]:
app = FaceAnalysis(name="buffalo_l")
app.prepare(ctx_id=0, det_size=(640, 640))


Enrollment

In [ ]:
print("📸 Look at the camera...")
img = take_photo()
cv2_imshow(img)

faces = app.get(img)

if len(faces) == 0:
    raise ValueError("❌ No face detected")

face = faces[0]
embedding = face.embedding

np.save("registered_face.npy", embedding)
print("✅ Face enrolled successfully!")


Testing

In [ ]:
print("📸 Testing... Look at the camera")
img = take_photo()
cv2_imshow(img)

faces = app.get(img)

if len(faces) == 0:
    raise ValueError("❌ No face detected")

test_embedding = faces[0].embedding
saved_embedding = np.load("registered_face.npy")

score = cosine_similarity(
    test_embedding.reshape(1, -1),
    saved_embedding.reshape(1, -1)
)[0][0]

print(f"🔍 Similarity score: {score:.3f}")

if score > 0.65:
    print("✅ SAME PERSON")
else:
    print("❌ DIFFERENT PERSON")
